# work log - database disagreement evaluation

# Table of Contents

- [Setup](#Setup)

    - [Setup - Imports](#Setup---Imports)
    - [Setup - Initialize Django](#Setup---Initialize-Django)

- [Work](#Work)
- [TODO](#TODO)

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
import csv
import datetime
import json
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

packages imported at 2017-07-05 18:06:54.223994


In [2]:
%pwd

'/home/jonathanmorgan/work/sourcenet/django/research/work/msu_phd_work'

In [3]:
%ls

2016.12.04-work_log.ipynb  reliability-assess_name_data.py
2016.12.09-work_log.ipynb  reliability-build_name_data.py
2016.12.10-work_log.ipynb  reliability_names_evaluations.txt
2016.12.11-work_log.ipynb  remove_single_names-work_log.ipynb
django_init.py             work_log-database_disagreement_evaluation.ipynb
LICENSE                    work_log-ground_truth-evaluate_disagreements.ipynb
README.md


## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

You need to have installed your virtualenv with django as a kernel, then select that kernel for this notebook.

In [4]:
%run django_init.py

django initialized at 2017-07-05 22:07:02.186291


Import any `sourcenet` or `sourcenet_analysis` models or classes.

In [5]:
# sourcenet_analysis models.
from sourcenet_analysis.models import Reliability_Names_Evaluation

print( "sourcenet and sourcenet_analysis packages imported at " + str( datetime.datetime.now() ) )

sourcenet and sourcenet_analysis packages imported at 2017-07-05 22:07:03.566530


# Work

- Back to [Table of contents](#Table-of-contents)

Read and parse delimited file of Reliability_Names rows we've arleady processed.  Create database row for each.

In [6]:
# CONSTANTS-ish

# header indexes:
#ID | Name | Article | Article_Data_List | Status | Error? (SHB = Should Have Been)| Notes
INDEX_RELIABILITY_NAMES_ID = 0
INDEX_PERSON_NAME = 1
INDEX_ARTICLE_INFO = 2
INDEX_ARTICLE_DATA_LIST = 3
INDEX_STATUS = 4
INDEX_STATUS_MESSAGE = 5
INDEX_NOTES = 6

# declare variables
delimited_file_path = ""
delimited_file = None
delimited_reader = None
current_row = None
rn_instance = None
rne_instance = None

# declare variables - column values
reliability_names_id = -1
person_name = ""
article_info = ""
article_data_list = ""
status = ""
status_message = ""
notes = ""
label = ""

# file path?
delimited_file_path = "reliability_names_evaluations.txt"

# read file
with open( delimited_file_path ) as delimited_file:
    
    # CSV reader
    delimited_reader = csv.reader( delimited_file, delimiter='|' )

    # first row is column names
    current_row = six.next( delimited_reader )
    
    # init shared values
    label = "prelim_month"
    
    # now loop over rest of rows.
    for current_row in delimited_reader:
        
        # print out the row.
        column_count = len( current_row )
        print( "==> row column count = " + str( column_count ) + " ( " + str( column_count == 7 ) + " ) - row: " + ",".join( current_row ) )
        
        # get values
        reliability_names_id = current_row[ INDEX_RELIABILITY_NAMES_ID ]
        reliability_names_id = int( reliability_names_id )
        person_name = current_row[ INDEX_PERSON_NAME ]
        article_info = current_row[ INDEX_ARTICLE_INFO ]
        article_data_list = current_row[ INDEX_ARTICLE_DATA_LIST ]
        status = current_row[ INDEX_STATUS ]
        status_message = current_row[ INDEX_STATUS_MESSAGE ]
        notes = current_row[ INDEX_NOTES ]
        
        # check to see if Reliability_Names record still exists.
        
        
        # Make Reliability_Names_Evaluation instance
        rne_instance = Reliability_Names_Evaluation.create_from_reliability_names( reliability_names_id,
                                                                                   label_IN = label,
                                                                                   status_IN = status,
                                                                                   status_message_IN = status_message,
                                                                                   notes_IN = notes )
        
    #-- END loop over rows. --#
    
#-- END with open( delimited_file_path ) --#

==> row column count = 7 ( True ) - row: 8362 , Jack Hagedorn , Article [20645](http://research.local/sourcenet/sourcenet/article/article_data/view_with_text/?article_id=20645) , Article_Data: [2322 (coder=8)](http://research.local/sourcenet/sourcenet/article/article_data/view/?article_id=20645&article_data_id_select=2322); [2984 (coder=2)](http://research.local/sourcenet/sourcenet/article/article_data/view/?article_id=20645&article_data_id_select=2984) , CORRECT , Mentioned, SHB Quoted , None
==> row column count = 7 ( True ) - row: 8367 , Bo Damstra , Article [20647](http://research.local/sourcenet/sourcenet/article/article_data/view_with_text/?article_id=20647) , Article_Data: [2355 (coder=10)](http://research.local/sourcenet/sourcenet/article/article_data/view/?article_id=20647&article_data_id_select=2355) , CORRECT , MISSED , None
==> row column count = 7 ( True ) - row: 8369 , Kyle Moody , Article [20647](http://research.local/sourcenet/sourcenet/article/article_data/view_with_te

DoesNotExist: Reliability_Names matching query does not exist.

# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- check if Reliability_Names exists.  If not, parse Article, Article_Data, Persons, and make a row by hand...  Or update the create() method so that it can accept a list of persons and Article_Datas instead of needing to get them from Reliability_Names.
- wipe table and re-run.
- go through and update by hand the flags for the other lists.
- make a list of those that were merged?
- update with single name removals?